In [149]:
import pandas as pd
import glob
import re
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from astropy.table import Table, Column

from sklearn.feature_extraction.text import CountVectorizer

In [224]:
dataset = 'query_text_16_12_29-052645-Copy1.txt'
lines = 1972870
test_lines = 2000
dataset_list = []

#line1 = dataset.readline()
#print(line1)
#line2 = dataset.readline()
#print(line2)
#full_line = [line1]
#print(full_line)


def read_dataset(file, line = 0):
    with open(file, 'r+', encoding = "utf8") as f:
        for i in range(0, line):
            dataset = f.readline()
            dataset_list.append(dataset)
        dataset_array = np.asarray(dataset_list)
        dataset_table = pd.DataFrame({"Individual Lines" : dataset_array})
        return(dataset_table)
        
#with open(file, 'r+', encoding = "utf8") as f:
 #   lines = f.readlines()    
    
    
    
table = read_dataset(dataset, test_lines)
table.head(3)

#print(dataset.readline().splitlines())
#print(dataset.readline().splitlines())
#print(dataset.readline().splitlines())

,Individual Lines
0,"059308\t""Sumerian: šeg6"", SLA // Studies Sigri..."
1,o. 1\t\t120 sa gi gu4 3 ba-ra-ceg6\t\n
2,o. 2\t\t´8` sa gi uzu ma-la-<ku> 2 ba-ra-ceg6\t\n


# Cleaning Ins.


Keep {} (brackets), - (dashes),
Remove [] (parantheses), '' (quotes/accents - damaged text) 
Remove the entire # comment and the ! 
Keep the "========"
c = š
C = capital(s) ^
Keep all the numbers 
Keep [...] damaged text


Keep the obverse and reverse 

In [225]:
def clean_dataset(dataset):
    for i in range(0, len(dataset)):
    # This code preforms the following:
    # Removing the []s, # and the subsequent comment, and !
    # Changing c = š and the same with Uppercase 
        cleaned_dataset = dataset
        cleaning_line = cleaned_dataset.loc[i, "Individual Lines"]
        if (type(re.match(r'\d{6}', cleaning_line)) == type(re.match(r'\d{6}', '012345'))):
            title_line = cleaning_line
            title_line = re.split('\t', cleaning_line)
            title_number = title_line[0]
            title_name = title_line[1]
            cleaned_dataset.drop(i, inplace = True)
        else:    
            delete_symbols_line = re.sub(r'\[{1,4}|\]{1,4}|!{1,4}', "", cleaning_line)
            changing_c_line = re.sub(r'c+', "š", delete_symbols_line)
            changing_C_line = re.sub(r'C+', 'Š', changing_c_line)
            remove_the_comments = re.sub(r'#.*$', '', changing_C_line)
            remove_the_tabs = re.sub(r'\s+', ' ', remove_the_comments)
            remove_the_quotes = re.sub(r"´|`", '', remove_the_tabs)
            cleaned_dataset.loc[i, 'Title Number'] = title_number
            cleaned_dataset.loc[i, 'Title Name'] = title_name
            cleaned_dataset.loc[i, "Individual Lines"] = remove_the_quotes
        #If it is preceded by a number immediately without a space keep C c
        #Forgot the remove the quotes - take them out
        #print(dataset)
    cleaned_dataset.index = range(len(cleaned_dataset))
    return(cleaned_dataset)

#
# Next time - Differentiate the title line from the text line C, c
# Deliminate the table by whitespace ("space" and "tab")
# Use "tab" and "newline" to create new lines 
# Dataset format: Title Number, Title ID, Individual Word, Individual Word Index, Upverse Reverse Identity
# For the title - it is not imperative to clean the Sumerian pieces just make sure you have the first 5 numbers
# We can go back and clean the title later. 
test = clean_dataset(table)
test.head(10)

,Individual Lines,Title Number,Title Name
0,o. 1 120 sa gi gu4 3 ba-ra-šeg6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."
1,o. 2 8 sa gi uzu ma-la-<ku> 2 ba-ra-šeg6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."
2,o. 3 x sa gi ša3-gar-du10 gu4 2 ba-ra-šeg6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."
3,o. 4 5+x sa gi bar šah2-niga 2 ba-ra-šeg6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."
4,o. 5 50 sa gi udu-niga 10 ba-ra-šeg6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."
5,o. 6 20 sa gi šah2-niga 2 ba-ra-šeg6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."
6,o. 7 x sa gi sila4 10 ba-ra-šeg6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."
7,r. 1 5 sa gi dar{mušen} 77 ba-ra-šeg6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."
8,r. 2 2 sa gi mušen-tur 20 ba-ra-šeg6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."
9,r. 3 u4 šakkan6-ne nam-erim2 in-ku5-ša,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /..."


In [249]:
# Creating function to clean data and to format it for lemmitization

def lemmitization_cleaning(dataset):
    columns = ["Title Number", "Title Name", "Op / Re Number", "Word", "Word Index", "Word Before", "Word After"]
    df_ = pd.DataFrame(columns=columns)
    df_ = df_.fillna(0)
    count = 0
    for i in range(0, len(dataset)):
        lines = dataset.loc[i, "Individual Lines"] 
        number = dataset.loc[i, "Title Number"]
        name = dataset.loc[i, "Title Name"]
        split = re.split("\s", lines)
        op_re_verse = split[0] + split[1]
        split = split[2:-1]
        num_words = len(split)
        for j in range(0, num_words):
            df_.loc[count,"Title Number"] = number
            df_.loc[count,"Title Name"] = name
            df_.loc[count,"Op / Re Number"] = op_re_verse
            df_.loc[count, "Word"] = split[j]
            df_.loc[count, "Word Index"] = j
            if num_words != 1:
                if j == 0:
                    df_.loc[count, "Word After"] = split[1]
                elif j > 0 and j < num_words-1:
                    df_.loc[count, "Word Before"] = split[j-1]
                    df_.loc[count, "Word After"] = split[j+1]
                else:
                    df_.loc[count, "Word Before"] = split[-1]
            count += 1
    return(df_)
    
        
test = lemmitization_cleaning(table)
test.head(100)

,Title Number,Title Name,Op / Re Number,Word,Word Index,Word Before,Word After
0,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.1,120,0,NaN,sa
1,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.1,sa,1,120,gi
2,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.1,gi,2,sa,gu4
3,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.1,gu4,3,gi,3
4,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.1,3,4,gu4,ba-ra-šeg6
5,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.1,ba-ra-šeg6,5,ba-ra-šeg6,NaN
6,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.2,8,0,NaN,sa
7,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.2,sa,1,8,gi
8,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.2,gi,2,sa,uzu
9,059308,"""Sumerian: šeg6"", SLA // Studies Sigrist 185 /...",o.2,uzu,3,gi,ma-la-<ku>


In [ ]:
# IDENTIFIERS - "We are looking for something"
# \d any number
# \D anything but a number
# \s space
# \S anything but a space
# \w any character
# \W anything but a character
# . any character, except for a newline (you write the character next to it)
# \b the whitespace around words 
# \. any period 

# Modifiers
# {1,3} we are expecting one to three of them \d{1,3} any number 1-3 in length
# + only if there is one or more of them
# ? might match 0 or 1
# * match 0 or more 
# $ match the end of string
# ^ matching the beginning of a string
# | either or \d{1-3} | \w{5-6}
# [] range or "variance" [A - Z] look in between A and Z
# Another example [1-5a-qA-Z] anything between 1-5, a-q, and then A-Z
# {x} expecting x amount 

# White Space Characters"
# \n new line
# \s space
# \t tab
# \e escape 
# \f form feed
# \r return 

#Don't forget that these are special characters:
# . + * ? [ ] $ ^ () {} | \ 

exampleString = '''
Jessica is 15 years old, and Daniel is 27 years old. 
Edward is 97, and his gradnfather, Oscar, is 102. 
'''

ages = re.findall(r'\d{1,3}', exampleString) # any number between 1-3 in length
names = re.sub(r'[A-Z][a-z]*', '', exampleString)

print(ages)
print(names)

In [211]:
columns = ['A','B', 'C']
df_ = pd.DataFrame(columns=columns)
df_
df_.loc[0, 'A'] = "Live"
df_


,A,B,C
0,Live,NaN,NaN
